# 회전교차로 상호작용 예측 모델 학습 (Colab)

SDD Death Circle 데이터셋을 사용한 이기종 교통 상호작용 예측 모델 학습

## 전체 프로세스 흐름

```
Phase 1: 환경 설정 (5분)
  ↓
Phase 2: 데이터 준비 (10분)
  ↓
Phase 3: 데이터 전처리 (30분)
  ↓
Phase 4: 모델 학습 (1-4시간) ⚡
  ↓
Phase 5: 모델 평가 (30분)
  ↓
Phase 6: 결과 다운로드 (5분)
```


In [ ]:
# 셀 1: 라이브러리 설치
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q torch-geometric torch-geometric-temporal
!pip install -q pandas numpy scipy scikit-learn matplotlib seaborn
!pip install -q opencv-python networkx tqdm pyyaml shapely tensorboard

import torch
print(f"✓ GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# 셀 2: 프로젝트 파일 업로드
# 방법 A: GitHub 클론
!git clone https://github.com/your-repo/Roundabout_AI.git
%cd Roundabout_AI

# 방법 B: 직접 업로드 (주석 해제)
# from google.colab import files
# uploaded = files.upload()
# !unzip project.zip -d Roundabout_AI
# %cd Roundabout_AI


In [ ]:
# 셀 3: 데이터셋 업로드
# 방법 A: Google Drive 마운트 (권장)
from google.colab import drive
drive.mount('/content/drive')

import os
os.symlink('/content/drive/MyDrive/Roundabout_AI/data', '/content/Roundabout_AI/data')

# 방법 B: 직접 업로드 (주석 해제)
# from google.colab import files
# # data/sdd/converted/*.csv 파일들 업로드


## Phase 2: 데이터 준비


In [ ]:
# 셀 4: 데이터 확인
import sys
sys.path.append('/content/Roundabout_AI')

from pathlib import Path
import pandas as pd
import numpy as np

data_dir = Path('/content/Roundabout_AI/data/sdd/converted')
csv_files = sorted(data_dir.glob('*.csv'))

print("=" * 60)
print("데이터 확인")
print("=" * 60)
print(f"✓ 변환된 데이터 파일: {len(csv_files)}개\n")

for f in csv_files:
    df = pd.read_csv(f)
    print(f"{f.name}:")
    print(f"  행 수: {len(df):,}")
    print(f"  트랙 수: {df['track_id'].nunique()}")
    print(f"  에이전트 타입: {sorted(df['agent_type'].unique())}")
    print()


## Phase 3: 데이터 전처리


In [ ]:
# 셀 5: 데이터 통합 및 전처리
from src.data_processing.preprocessor import TrajectoryPreprocessor
import pickle

# 데이터 통합
all_data = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df['video_id'] = csv_file.stem.replace('_converted', '')
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)
print(f"✓ 통합 데이터: {len(combined_df):,}행")

# 전처리
preprocessor = TrajectoryPreprocessor(
    obs_window=30,
    pred_window=50,
    sampling_rate=10.0
)

# 샘플링 (전체 데이터가 많을 경우)
sample_ratio = 0.3
unique_tracks = combined_df['track_id'].unique()
sampled_tracks = np.random.choice(
    unique_tracks,
    size=int(len(unique_tracks) * sample_ratio),
    replace=False
)
sampled_df = combined_df[combined_df['track_id'].isin(sampled_tracks)]

# 윈도우 생성
windows = []
for track_id in sampled_df['track_id'].unique():
    track_data = sampled_df[sampled_df['track_id'] == track_id].sort_values('frame_id')
    if len(track_data) >= 80:
        track_windows = preprocessor.create_sliding_windows(track_data)
        windows.extend(track_windows)

print(f"✓ 생성된 윈도우: {len(windows):,}개")

# 저장
output_dir = Path('/content/Roundabout_AI/data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

with open(output_dir / 'sdd_windows.pkl', 'wb') as f:
    pickle.dump(windows, f)

print(f"✓ 저장 완료: {output_dir / 'sdd_windows.pkl'}")


## Phase 4: 모델 학습


In [ ]:
# 셀 6: 데이터 로더 생성
from src.training.data_loader import TrajectoryDataset, create_dataloader, split_dataset

# 데이터 분할
train_windows, val_windows, test_windows = split_dataset(
    windows,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

train_dataset = TrajectoryDataset(train_windows)
val_dataset = TrajectoryDataset(val_windows)

train_loader = create_dataloader(train_dataset, batch_size=32, shuffle=True)
val_loader = create_dataloader(val_dataset, batch_size=32, shuffle=False)

print(f"✓ 학습 데이터: {len(train_windows):,}개")
print(f"✓ 검증 데이터: {len(val_windows):,}개")


In [ ]:
# 셀 7: 모델 생성 및 학습
import torch
from src.models.a3tgcn_model import create_a3tgcn_model
from src.training.trainer import create_trainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

# 모델 생성
model = create_a3tgcn_model(
    node_features=9,
    hidden_channels=64,
    pred_steps=50,
    use_map=False
)

print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

# Trainer 생성
config = {
    'optimizer': 'adam',
    'learning_rate': 0.001,
    'weight_decay': 1e-5,
    'scheduler': 'reduce_on_plateau',
    'loss': 'mse',
    'num_epochs': 50,
    'early_stopping': {'patience': 10, 'min_delta': 0.001},
    'log_dir': '/content/Roundabout_AI/runs',
    'save_dir': '/content/Roundabout_AI/checkpoints',
    'max_grad_norm': 1.0
}

trainer = create_trainer(model, train_loader, val_loader, config)

# 학습 시작
print("\n" + "=" * 80)
print("학습 시작")
print("=" * 80)

trainer.train(50)


In [ ]:
# 셀 8: TensorBoard (별도 탭에서 확인)
%load_ext tensorboard
%tensorboard --logdir /content/Roundabout_AI/runs --port 6006


## Phase 5: 모델 평가


In [ ]:
# 셀 9: 모델 평가
from src.evaluation.evaluator import ModelEvaluator

evaluator = ModelEvaluator(model, device)
metrics = evaluator.evaluate_dataset(val_loader, max_batches=50)

print("=" * 60)
print("평가 결과")
print("=" * 60)
for key, value in metrics.items():
    if 'Rate' in key:
        print(f"{key:25s}: {value:.4f} ({value*100:.2f}%)")
    else:
        print(f"{key:25s}: {value:.4f} m")


In [ ]:
# 셀 10: 결과 다운로드
from google.colab import files
from pathlib import Path

# 체크포인트 다운로드
checkpoint_path = Path('/content/Roundabout_AI/checkpoints/best_model.pth')
if checkpoint_path.exists():
    files.download(str(checkpoint_path))
    print("✓ 모델 다운로드 완료")
